In [1]:
import pandas as pd
import numpy as np
import pickle
import patsy

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
df = pd.read_csv('loan.csv')

/home/brendanfitzpatrick/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_key = pd.read_excel('LCDataDictionary.xlsx')
df_key.set_index('LoanStatNew')

,Description
LoanStatNew,
addr_state,The state provided by the borrower in the loan application
annual_inc,The self-reported annual income provided by the borrower during registration.
annual_inc_joint,The combined self-reported annual income provided by the co-borrowers during registration
application_type,Indicates whether the loan is an individual application or a joint application with two co-borrowers
collection_recovery_fee,post charge off collection fee
collections_12_mths_ex_med,Number of collections in 12 months excluding medical collections
delinq_2yrs,The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years
desc,Loan description provided by the borrower
dti,"A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income."


&nbsp;
### Remove sub policy and current loans
&nbsp;

In [5]:
df = df[(df.loan_status == 'Fully Paid')
                | (df.loan_status == 'Charged Off')
                | (df.loan_status == 'Default')]

&nbsp;
### Combine 'Charged Off' and 'Default'
&nbsp;

In [6]:
df['loan_status'] = df.loan_status.apply(lambda x: 'Fully Paid' 
                                              if x == 'Fully Paid' 
                                              else 'Default')

In [7]:
df.loan_status.unique()

array(['Fully Paid', 'Default'], dtype=object)

&nbsp;
### Separate Lending Club's Allocation
&nbsp;

In [8]:
df['lc_allocation'] = df.loan_amnt - df.funded_amnt_inv

In [9]:
df['term'] = pd.to_numeric(df.term.str.strip().str.replace('months',''))

&nbsp;
### Remove NaN's
&nbsp;

In [10]:
df = df[df.revol_util.notnull()]

&nbsp;
### Create Dummy Variables
&nbsp;

In [11]:
def yes_no_binary(x):
    if (x=='y'):
        return 1
    if (x=='n'):
        return 0

In [12]:
df['pymnt_plan'] = df['pymnt_plan'].apply(yes_no_binary)

In [13]:
def whole_fractional_binary(x):
    if (x=='w'):
        return 1
    if (x=='f'):
        return 0

In [14]:
df['initial_list_status'] = df['initial_list_status'].apply(whole_fractional_binary)

In [15]:
def event_last_year(x):
    if (x < 12*1):
        return 1
    else:
        return 0

In [16]:
def event_last_two_years(x):
    if (x < 12*2):
        return 1
    else:
        return 0

In [17]:
def event_last_five_years(x):
    if (x < 12*5):
        return 1
    else:
        return 0

In [18]:
df['major_derog_within_1'] = df.mths_since_last_major_derog.apply(event_last_year)
df['major_derog_within_2'] = df.mths_since_last_major_derog.apply(event_last_two_years)
df['major_derog_within_5'] = df.mths_since_last_major_derog.apply(event_last_five_years)

In [19]:
df['record_within_1'] = df.mths_since_last_record.apply(event_last_year)
df['record_within_2'] = df.mths_since_last_record.apply(event_last_two_years)
df['record_within_5'] = df.mths_since_last_record.apply(event_last_five_years)

In [20]:
df['emp_length'].unique()

array(['10+ years', '< 1 year', '3 years', '9 years', '4 years', '5 years',
       '1 year', '6 years', '2 years', '7 years', '8 years', 'n/a'], dtype=object)

In [21]:
def emp_length_10_map(x):
    if(x == '10+ years'):
        return 1
    else:
        return 0

In [22]:
def emp_length_5_map(x):
    if(x == '10+ years' or x == '9 years' or x == '8 years' 
       or x == '7 years' or x == '6 years' or x == '5 years'):
        return 1
    else:
        return 0

In [23]:
def emp_length_1_map(x):
    if(x == '< 1 year' or x == 'n/a'):
        return 0
    else:
        return 1

In [24]:
df['emp_length_greater_1_yr'] = df.emp_length.apply(emp_length_1_map)
df['emp_length_greater_5_yrs'] = df.emp_length.apply(emp_length_5_map)
df['emp_length_greater_10_yrs'] = df.emp_length.apply(emp_length_10_map)

In [25]:
df['earliest_cr_line'] = pd.to_datetime(df.earliest_cr_line,format='%b-%Y')
df['issue_d'] = pd.to_datetime(df.issue_d,format='%b-%Y')

In [26]:
df['earliest_cr_line_delta'] = (df.issue_d - df.earliest_cr_line) / np.timedelta64(1, 'M')

In [27]:
grades_df = patsy.dmatrix('grade',data=df,return_type='dataframe')
grades_columns = ['grade_intercept',
                  'B_grade',
                  'C_grade',
                  'D_grade',
                  'E_grade',
                  'F_grade',
                  'G_grade']
grades_df.columns = grades_columns

In [28]:
df = df.join(grades_df)

In [29]:
home_ownership_df = patsy.dmatrix('home_ownership',data=df,return_type='dataframe')
home_ownership_columns = ['home_ownership_intercept',
                          'home_ownership_mortgage',
                          'home_ownership_none',
                          'home_ownership_other',
                          'home_ownership_own',
                          'home_ownership_rent']
home_ownership_df.columns = home_ownership_columns

In [30]:
df = df.join(home_ownership_df)

In [31]:
verification_df = patsy.dmatrix('verification_status',data=df,return_type='dataframe')
verification_columns = ['verification_intercept',
                        'source_verified',
                        'verified']
verification_df.columns = verification_columns

In [32]:
df = df.join(verification_df)

In [33]:
"""emp_length_df = patsy.dmatrix('emp_length',data=df,return_type='dataframe')
emp_length_columns = ['emp_length_intercept', 
                      'emp_length_greater_10_yrs',
                      'emp_length_2_yrs',
                      'emp_length_3_yrs',
                      'emp_length_4_yrs',
                      'emp_length_5_yrs',
                      'emp_length_6_yrs',
                      'emp_length_7_yrs',
                      'emp_length_8_yrs',
                      'emp_length_9_yrs', 
                      'emp_length_less_1_yr',
                      'emp_length_n/a]']
emp_length_df.columns = emp_length_columns"""

"emp_length_df = patsy.dmatrix('emp_length',data=df,return_type='dataframe')\nemp_length_columns = ['emp_length_intercept', \n                      'emp_length_greater_10_yrs',\n                      'emp_length_2_yrs',\n                      'emp_length_3_yrs',\n                      'emp_length_4_yrs',\n                      'emp_length_5_yrs',\n                      'emp_length_6_yrs',\n                      'emp_length_7_yrs',\n                      'emp_length_8_yrs',\n                      'emp_length_9_yrs', \n                      'emp_length_less_1_yr',\n                      'emp_length_n/a]']\nemp_length_df.columns = emp_length_columns"

In [34]:
"""df = df.join(emp_length_df)"""

'df = df.join(emp_length_df)'

BF NOTE: Try outstanding principal LC allocation if in need of additional features

&nbsp;
### Remove columns with high NaNs
&nbsp;

In [35]:
df = df.drop(['open_acc_6m',
              'open_il_6m',
              'open_il_12m',
              'open_il_24m',
              'mths_since_rcnt_il',
              'total_bal_il',
              'il_util',
              'open_rv_12m',
              'open_rv_24m',
              'max_bal_bc',
              'all_util',
              'inq_fi',
              'total_cu_tl',
              'inq_last_12m'], 1)

### Group seperate DataFrames

&nbsp;

Create three seperate dataframes for:
  * Individual loans
  * Joint loans
  [* Loans that do not meet Lending Club's credit policy]
  
&nbsp;
  

In [36]:
"""df_sub_policy = df[(df.loan_status == 'Does not meet the credit policy. Status:Fully Paid') 
                   | (df.loan_status == 'Does not meet the credit policy. Status:Charged Off')]"""

"df_sub_policy = df[(df.loan_status == 'Does not meet the credit policy. Status:Fully Paid') \n                   | (df.loan_status == 'Does not meet the credit policy. Status:Charged Off')]"

&nbsp;
### Remove sub-policy loans
&nbsp;

In [37]:
df = df[(df.loan_status != 'Does not meet the credit policy. Status:Fully Paid') & (df.loan_status != 'Does not meet the credit policy. Status:Charged Off')]

In [38]:
df_joint = df[df.application_type=='JOINT']

In [39]:
df_inv = df[df.application_type=='INDIVIDUAL']

In [40]:
df.application_type.unique()

array(['INDIVIDUAL', 'JOINT'], dtype=object)

In [41]:
df_inv.drop('annual_inc_joint',1)
df_inv.drop('dti_joint',1)
df_inv.drop('verification_status_joint',1)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,G_grade,home_ownership_intercept,home_ownership_mortgage,home_ownership_none,home_ownership_other,home_ownership_own,home_ownership_rent,verification_intercept,source_verified,verified
0,1077501,1296599,5000.0,5000.0,4975.000000,36,10.65,162.87,B,B2,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1077430,1314167,2500.0,2500.0,2500.000000,60,15.27,59.83,C,C4,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
2,1077175,1313524,2400.0,2400.0,2400.000000,36,15.96,84.33,C,C5,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,1076863,1277178,10000.0,10000.0,10000.000000,36,13.49,339.31,C,C1,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
5,1075269,1311441,5000.0,5000.0,5000.000000,36,7.90,156.46,A,A4,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
7,1072053,1288686,3000.0,3000.0,3000.000000,36,18.64,109.43,E,E1,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
8,1071795,1306957,5600.0,5600.0,5600.000000,60,21.28,152.39,F,F2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
9,1071570,1306721,5375.0,5375.0,5350.000000,60,12.69,121.45,B,B5,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
10,1070078,1305201,6500.0,6500.0,6500.000000,60,14.65,153.45,C,C3,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
11,1069908,1305008,12000.0,12000.0,12000.000000,36,12.69,402.54,B,B5,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [42]:
with open('df_inv.pkl', 'wb') as picklefile:
    pickle.dump(df_inv, picklefile)

In [43]:
with open('df_joint.pkl', 'wb') as picklefile:
    pickle.dump(df_joint, picklefile)

In [49]:
df_both = df[(df.application_type=='JOINT') | (df.application_type=='INDIVIDUAL')]

In [50]:
with open('df_both.pkl', 'wb') as picklefile:
    pickle.dump(df_both, picklefile)

In [44]:
"""with open('df_sub_policy.pkl', 'wb') as picklefile:
    pickle.dump(df_sub_policy, picklefile)"""

"with open('df_sub_policy.pkl', 'wb') as picklefile:\n    pickle.dump(df_sub_policy, picklefile)"

In [45]:
!ls -l

total 963380
-rw-rw-r-- 1 brendanfitzpatrick brendanfitzpatrick     22092 Aug  2 09:37 03-mcnulty_brendan_choropleths.ipynb
-rw-r--r-- 1 brendanfitzpatrick brendanfitzpatrick     43418 Aug  2 09:52 03_mcnulty_brendan_cleaning.ipynb
-rw-rw-r-- 1 brendanfitzpatrick brendanfitzpatrick      8890 Aug  2 09:50 03-mcnulty_brendan_donuts.ipynb
-rw-rw-r-- 1 brendanfitzpatrick brendanfitzpatrick      6552 Aug  1 22:00 03-mcnulty_brendan_flask_model.ipynb
-rw-rw-r-- 1 brendanfitzpatrick brendanfitzpatrick     78461 Jul 30 03:41 03_mcnulty_brendan_modeling-Copy1.ipynb
-rw-rw-r-- 1 brendanfitzpatrick brendanfitzpatrick     68462 Aug  1 22:31 03_mcnulty_brendan_modeling.ipynb
-rwxr-xr-x 1 brendanfitzpatrick brendanfitzpatrick    172221 Jul 28 21:30 challenge_set_1_brendan.ipynb
-rw-r--r-- 1 brendanfitzpatrick brendanfitzpatrick    178337 Jul 28 15:12 challenge_set_7_brendan.ipynb
-rw-rw-r-- 1 brendanfitzpatrick brendanfitzpatrick     28379 Jul 28 20:40 challenge_set_8_brendan.ipynb
-rw-rw-r-- 1 bren